# Parcial 3 Métodos Númericos

__Valentina Galindo Peña__

*__ID__*: 000484044

__Juan David Parra Cantor__

*__ID__*: 000324320

## Tema B

El planteamiento de la ecuación diferencial es:

$$y'' = 2y' - y + xe^x - x$$

con condiciones de frontera:

$$-y(0) = 0$$

$$y(2) = -4$$

La solución exacta es:

$$y(x) = \frac{1}{6}x^3e^x - \frac{5}{3}xe^x + 2e^x - x - 2$$

Para implementar las diferencias finitas, necesitamos discretizar la ecuación diferencial. La discretización de la segunda derivada en diferencias finitas es:

$$y''(x_i) \approx \frac{y(x_{i+1}) - 2y(x_i) + y(x_{i-1})}{h^2}$$

donde $h$ es el tamaño del paso.

La discretización de la primera derivada en diferencias finitas es:

$$y'(x_i) \approx \frac{y(x_{i+1}) - y(x_{i-1})}{2h}$$

Sustituyendo estas expresiones en la ecuación diferencial, obtenemos:

$$\frac{y(x_{i+1}) - 2y(x_i) + y(x_{i-1})}{h^2} = 2\frac{y(x_{i+1}) - y(x_{i-1})}{2h} - y(x_i) + x_ie^{x_i} - x_i$$

Simplificando y reordenando términos, obtenemos:

$$y(x_{i+1}) - 2y(x_i) + y(x_{i-1}) = h^2(2\frac{y(x_{i+1}) - y(x_{i-1})}{2h} - y(x_i) + x_ie^{x_i} - x_i)$$

$$y(x_{i+1}) - 2y(x_i) + y(x_{i-1}) = h(2y(x_{i+1}) - 2y(x_{i-1}) - 3hy(x_i) + hx_ie^{x_i} - hx_i)$$

Ahora, podemos construir la matriz A y el vector b utilizando estas expresiones.

## Librerias

## Ecuaciones y condiciones de frontera

## Solucion exacta 

## Diferencias finitas

## Graficas 